- reference
    - https://github.com/HMJiangGatech/pytorch-pretrained-BERT/blob/master/examples/lm_finetuning/pregenerate_training_data.py
    - https://paul-hyun.github.io/bert-01/

In [121]:
# vocab loading
import sentencepiece as spm

VOCAB_PATH = "/home/henry/Documents/wrapper/source"
vocab_file = f"{VOCAB_PATH}/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [122]:
from types import SimpleNamespace
import torch.nn as nn
import torch.nn.functional as F
config = dict({
    "n_enc_vocab": len(vocab),
    "n_enc_seq": 256,
    "n_seg_type": 2,
    "n_layer": 6,
    "d_hidn": 256,
    "i_pad": 0,
    "d_ff": 1024,
    "n_head": 4,
    "d_head": 64,
    "dropout": 0.1,
    "layer_norm_epsilon": 1e-12
})
config = SimpleNamespace(**config)

In [123]:
print(config)

namespace(d_ff=1024, d_head=64, d_hidn=256, dropout=0.1, i_pad=0, layer_norm_epsilon=1e-12, n_enc_seq=256, n_enc_vocab=8007, n_head=4, n_layer=6, n_seg_type=2)


# BERT

In [124]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [11]:
class BERT(nn.Module):
    """
    outputs: [bs, len_seq, d_hidn] <- 잘 임베딩된 input seq
    ointput_cls = outputs[:, 0].contiguous(): [bs, d_hidn]
    - classification은 [cls] token의 임베딩만 사용

    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.encoder = Encoder(self.config)
        self.linear = nn.Linear(config.d_hidn, config.d_hidn)
        self.activation = torch.tanh
    
    def forward(slef, inputs, segment):
        outputs, self_attn_probs = self.encoder(inputs, segments)
        ointput_cls = outputs[:, 0].contiguous()
        outputs_cls = self.linear(outputs_cls)
        outputs_cls = self.activation(outputs_cls)
        return outputs, outputs_cls, self_attn_probs
    
    def save(self, epoch, loss, path):
        torch.save({
            'epoch': epoch,
            'loss': loss,
            'state_dict': self.state_dict()
                   }, path)
        
    def load(self, path):
        save = torch.load(path)
        self.load_state_dict(save['state_dict'])
        return save['epoch'], save['loss']

In [12]:
class BERTpretrain(nn.Module):
    """
    self.feedforward_lm.weight
    - transformer encoder의 pretrained embedding layer weight 사용
    logits_cls: [bs, 2]
    - binary classification
    logits_lm: [bs, len_enc_seq, n_enc_vocab]
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.bert  = BERT(self.config)
        # cls
        self.feedforward_cls = nn.Linear(self.config.d_hidn, 2, bias=False)
        # lm
        self.feedforward_lm = nn.Linear(self.config.d_hidn, self.config.n_enc_vocab, bias=False)
        self.feedforward_lm.weight = self.bert.encoder.enc_emb.weight
    
    def forward(self, inputs, segments):
        outputs, outputs_cls, attn_probs = self.bert(inputs, segments)
        logits_cls = self.feedforward_cls(outputs)
        logits_lm = self.feedforward_lm(outputs)
        return logits_cls, logits_lm, attn_probs

# Masking
- https://github.com/codertimo/BERT-pytorch/blob/master/bert_pytorch/trainer/pretrain.py

In [120]:
def create_pretrain_mask(tokens, mask_cnt, vocab_list):
    """
    masking subwords(15% of entire subwords)
    - mask_cnt: len(subwords) * 0.15
    - [MASK]: 80% of masking candidate token
    - original token: 10% of masking candidate token
    - another token: 10% of masking candidate token
    """
    candidate_idx = []

    ## subwords in the same list augment a sementic word 
    ## eg. [[0], [1], [2], [4, 5]] -> token_idx 4 + 5 is semantic word
    # A list represent a sementic word
    for i, token in enumerate(tokens):
        if token == '[CLS]' or token == '[SEP]':
            continue
        if 0 < len(candidate_idx) and token.find(u'\u2581') < 0: #  LOWER ONE EIGHTH BLOCK
#        if 0 < len(candidate_idx) and token.find('_') < 0: #  test code
            candidate_idx[-1].append(i)
        else:
            candidate_idx.append([i])
    np.random.shuffle(candidate_idx)

    mask_lms = []
    for idx_set in candidate_idx:
        # check if len(mask_lms) exceeds threshold
        if len(mask_lms) >= mask_cnt:
            break
        if len(mask_lms) + len(idx_set) > mask_cnt:
            continue

        ## masking subwords with 15% probability
        ## mask_cnt is len(subwords) * 0.15 
        # iter subwords idx
        for sub_idx in idx_set:
            masked_token = None

            ### assign value to masked token: [MASK], original token, random token
            # 80% of masking candidate are replaced with '[MASK]' token
            if np.random.uniform() < 0.8:
                masked_token = '[MASK]'
            # remainng 20% of masking candidate
            else:
                # 10% of remaining preserve original token
                if np.random.uniform() < 0.5:
                    masked_token = tokens[sub_idx]
                # 10% of ones are replaced with rnadom token    
                else:
                    masked_token = np.random.choice(vocab_list)

                ### replace subword with masked_token value    
                mask_lms.append({'idx': sub_idx, 'label':tokens[sub_idx]})
                tokens[sub_idx] = masked_token
                
    mask_lms = sorted(mask_lms, key=lambda x: x['idx'])
    mask_idx = [mask_dict['idx'] for mask_dict in mask_lms]
    mask_label = [mask_dict['label'] for mask_dict in mask_lms]
#     print(candidate_idx)
#     print(mask_lms)
    return tokens, mask_idx, mask_label

In [56]:
def truncate_token(tokenA, tokenB, max_seq):
    """
    truncate long sequence
    """
    while True:
        total_len = len(tokenA) + len(tokenB)
        print('max token {}\ntotal_len {} = {} + {}'.format(max_seq, total_len, len(tokenA), len(tokenB)))
        if total_len <= max_seq:
            break
        if len(tokenA) > len(tokenB):
            tokenA.pop()
        else:
            tokenB.pop()

- example code

In [16]:
vocab_list = ['_I am', '_on', '_it', '_so wh', 'at',]
tokens = ['_I am', '_on', '_it', '[CLS]', '_so wh', 'at',]
mask_cnt = int(len(tokens) * 0.15)+1
create_pretrain_mask(tokens, mask_cnt, vocab_list)

---
# pretrain dataset for each paragraph

In [175]:
in_PATH = '/home/henry/Documents/wrapper/source/kowiki_sample.txt' 
out_PATH = '/home/henry/Documents/wrapper/source/out_kowiki_sample' + '_{}.json'

count = 1
n_seq = 10
mask_prob = 0.15

make_pretrain_data(vocab, in_PATH, out_PATH, count, n_seq, mask_prob)

[[['▁지', '미', '▁카', '터'], ['▁제임스', '▁얼', '▁"', '지', '미', '"', '▁카', '터', '▁주', '니어', '(,', '▁192', '4', '년', '▁10', '월', '▁1', '일', '▁~', '▁)', '는', '▁민주', '당', '▁출신', '▁미국', '▁3', '9', '번째', '▁대통령', '▁(19', '7', '7', '년', '▁~', '▁1981', '년', ')', '이다', '.'], ['▁지', '미', '▁카', '터', '는', '▁조지', '아', '주', '▁섬', '터', '▁카', '운', '티', '▁플', '레', '인', '스', '▁마을', '에서', '▁태어났다', '.', '▁조지', '아', '▁공', '과', '대학교', '를', '▁졸업', '하였다', '.', '▁그', '▁후', '▁해', '군에', '▁들어가', '▁전', '함', '·', '원', '자', '력', '·', '잠', '수', '함', '의', '▁승', '무', '원으로', '▁일', '하였다', '.', '▁195', '3', '년', '▁미국', '▁해군', '▁대', '위로', '▁예', '편', '하였고', '▁이후', '▁땅', '콩', '·', '면', '화', '▁등을', '▁가', '꿔', '▁많은', '▁돈', '을', '▁벌', '었다', '.', '▁그의', '▁별', '명이', '▁"', '땅', '콩', '▁농', '부', '"', '▁(', 'P', 'e', 'an', 'ut', '▁F', 'ar', 'm', 'er', ')', '로', '▁알려', '졌다', '.'], ['▁196', '2', '년', '▁조지', '아', '▁주', '▁상', '원', '▁의원', '▁선거', '에서', '▁낙', '선', '하나', '▁그', '▁선거', '가', '▁부정', '선거', '▁', '였', '음을', '▁입', '증', '하게', '▁되어', '▁당선', 

In [171]:
def create_pretrain_instances(paragraph_ls, paragraph_idx, paragraph, n_seq, mask_prob, vocab_list):
    """
    create NSP train set
    """
    # 3 special token: [CLS], [SEP] for sent A, [SEP] for sent B
    max_seq_len = n_seq - 2 - 1
    target_seq_len = max_seq_len # [CLS], segmentA, segmentA, ..., [SEP], segmentB, segmentB, ...

    instances = []
    temp_sentence = []
    temp_sent_seq_length = 0 # num of tokens

    max_num_tokens = 256
    target_seq_len = np.random.randint(2, max_num_tokens) # min len of tokens
    for i, sent in enumerate(paragraph):
        ## A. not the last sentence of the paragraph
        temp_sentence.append(sent)
        temp_sent_seq_length += len(sent)

        ## B. check if it is the last sentence of the paragraph
        ## or temp_sent_seq_length is longer than or equal to target_seq_len
        if i == len(paragraph) - 1 or temp_sent_seq_length >= target_seq_len:
            if temp_sentence:
                ## A. sentence A segment: from 0 to a_end
                a_end = 1
                if len(temp_sentence) != 1:
                    a_end = np.random.randint(1, len(temp_sentence))
                # append the sentences to tokenA 
                # from the front to the back
                tokenA = []
                for _, s in enumerate(temp_sentence[:a_end]):
                    tokenA.extend(s)

                ## B. sentence B segment
                tokenB = []
                # A. Actual next
                if len(temp_sentence) > 1 and np.random.uniform() >= 0.5:
                    is_next = True
                    for j in range(a_end, len(temp_sentence)):
                        tokenB.extend(temp_sentence[j])
                # B. random next
                else:
                    is_next = False
                    tokenB_len = target_seq_len - len(tokenA)
                    random_para_idx = para_idx
                    while para_idx == random_para_idx:
                        random_para_idx = np.random.randint(0, len(paragraph_ls))
                    random_para = paragraph[random_para_idx]

                    random_start = np.random.randint(0, len(random_para))
                    for j in range(random_start, len(random_para)):
                        tokenB.extend(random_para[j])

                truncate_token(tokenA, tokenB, max_seq)
                assert 0 < len(tokenA)
                assert 0 < len(tokenB)

                tokens = ["[CLS]"] + tokenA + ["[SEP]"] + tokenB + ["[SEP]"]
                segment = [0]*(len(tokenA)  + 2) + [1]*(len(tokenB) + 1)
                
                tokens, mask_idx, mask_label = \
                    create_pretrain_mask(tokens, int((len(tokens)-3)*mask_prob), vocab_list)
                instance = {
                    'tokens': tokens,
                    'segment': segment,
                    'is_next': is_next,
                    'mask_idx': mask_idx,
                    'mask_label': mask_label
                }

                instances.append(instance)

            # reset segment candidate
            temp_sentence = []
            temp_sent_seq_length = 0
    
    return instances

- example code

In [11]:
para_idx = 1
paragraph = paragraph_ls[para_idx]
print(len(paragraph), paragraph[1])

6 ['▁카', '터', '는', '▁1970', '년대', '▁후반', '▁당시', '▁대한민국', '▁등', '▁인', '권', '▁후', '진', '국의', '▁국민', '들의', '▁인', '권을', '▁지', '키', '기', '▁위해', '▁노력', '했으며', ',', '▁취임', '▁이후', '▁계속', '해서', '▁도', '덕', '정', '치를', '▁내', '세', '웠다', '.']


- i = 0
    - add 1st sequence to meet target sequence length
    - max_len_doc == 8

In [126]:
n_seq = 256
max_seq = n_seq - 3
target_seq_len = max_seq

instances = []
temp_sentences = []
temp_sent_seq_length = 0
print('target seq len:', target_seq_len)

target seq len: 253


In [127]:
# the case of 1st sentence in 2nd paragraph 
para_idx = 1
paragraph = paragraph_ls[para_idx] # 2nd paragraph

temp_sentences = []
i = 0 
temp_sentences.append(paragraph[i]) # 1st sentence
temp_sent_seq_length += len(paragraph[i])

print('current seq length:', temp_sent_seq_length)
print('# of sentences in the paragraph: {}'.format(len(paragraph)))
print('temp_sentences len: {}'.format(len(temp_sentences)), '\n', temp_sentences)

if i == len(paragraph) - 1 or temp_sent_seq_length >= target_seq_len:
    print('Is it the last sentence of the paragraph \nor current chunk longer than target_seq_len?', True)

current seq length: 50
# of sentences in the paragraph: 6
temp_sentences len: 1 
 [['▁그러나', '▁이것은', '▁공', '화', '당', '과', '▁미국의', '▁유대', '인', '▁단', '체의', '▁반', '발', '을', '▁일으', '켰', '다', '.', '▁1979', '년', '▁백', '악', '관', '에서', '▁양', '국', '▁간의', '▁평화', '조', '약', '으로', '▁이끌', '어졌다', '.', '▁또한', '▁소련', '과', '▁제', '2', '차', '▁전략', '▁무', '기', '▁제한', '▁협', '상에', '▁조', '인', '했다', '.']]


- i = 1
    - add one more sentece to segments A to meet target sequence length

In [28]:
i = 1
temp_sentences.append(paragraph[i]) # 1st sentence
temp_sent_seq_length += len(paragraph[i])

print('current seq length:', temp_sent_seq_length)
print('# of sentences in the paragraph: {}'.format(len(paragraph)))
print('temp_sentences len: {}'.format(len(temp_sentences)), '\n', temp_sentences)

if i == len(paragraph) - 1 or temp_sent_seq_length >= target_seq_len:
    print('Is it the last sentence of the paragraph \nor current chunk longer than target_seq_len?', True)

current seq length: 87
# of sentences in the paragraph: 6
temp_sentences len: 2 
 [['▁그러나', '▁이것은', '▁공', '화', '당', '과', '▁미국의', '▁유대', '인', '▁단', '체의', '▁반', '발', '을', '▁일으', '켰', '다', '.', '▁1979', '년', '▁백', '악', '관', '에서', '▁양', '국', '▁간의', '▁평화', '조', '약', '으로', '▁이끌', '어졌다', '.', '▁또한', '▁소련', '과', '▁제', '2', '차', '▁전략', '▁무', '기', '▁제한', '▁협', '상에', '▁조', '인', '했다', '.'], ['▁카', '터', '는', '▁1970', '년대', '▁후반', '▁당시', '▁대한민국', '▁등', '▁인', '권', '▁후', '진', '국의', '▁국민', '들의', '▁인', '권을', '▁지', '키', '기', '▁위해', '▁노력', '했으며', ',', '▁취임', '▁이후', '▁계속', '해서', '▁도', '덕', '정', '치를', '▁내', '세', '웠다', '.']]


- i = 2

In [30]:
i = 2
temp_sentences.append(paragraph[i]) # 1st sentence
temp_sent_seq_length += len(paragraph[i])

print('current seq length:', temp_sent_seq_length)
print('# of sentences in the paragraph: {}'.format(len(paragraph)))
print('temp_sentences len: {}'.format(len(temp_sentences)), '\n', temp_sentences)

if i == len(paragraph) - 1 or temp_sent_seq_length >= target_seq_len:
    print('Is it the last sentence of the paragraph \nor current chunk longer than target_seq_len?', True)

current seq length: 161
# of sentences in the paragraph: 6
temp_sentences len: 3 
 [['▁그러나', '▁이것은', '▁공', '화', '당', '과', '▁미국의', '▁유대', '인', '▁단', '체의', '▁반', '발', '을', '▁일으', '켰', '다', '.', '▁1979', '년', '▁백', '악', '관', '에서', '▁양', '국', '▁간의', '▁평화', '조', '약', '으로', '▁이끌', '어졌다', '.', '▁또한', '▁소련', '과', '▁제', '2', '차', '▁전략', '▁무', '기', '▁제한', '▁협', '상에', '▁조', '인', '했다', '.'], ['▁카', '터', '는', '▁1970', '년대', '▁후반', '▁당시', '▁대한민국', '▁등', '▁인', '권', '▁후', '진', '국의', '▁국민', '들의', '▁인', '권을', '▁지', '키', '기', '▁위해', '▁노력', '했으며', ',', '▁취임', '▁이후', '▁계속', '해서', '▁도', '덕', '정', '치를', '▁내', '세', '웠다', '.'], ['▁그러나', '▁주', '▁이란', '▁미국', '▁대사', '관', '▁인', '질', '▁사건', '에서', '▁인', '질', '▁구', '출', '▁실패', '를', '▁이유로', '▁1980', '년', '▁대통령', '▁선거', '에서', '▁공', '화', '당의', '▁로', '널', '드', '▁레이', '건', '▁후보', '에게', '▁', '져', '▁결국', '▁재', '선에', '▁실패', '했다', '.', '▁또한', '▁임', '기', '▁말', '기에', '▁터', '진', '▁소련', '의', '▁아', '프가', '니', '스탄', '▁침공', '▁사건', '으로', '▁인해', '▁1980', '년', '▁하계', '▁올림픽', '에', '▁반',

In [31]:
i = 3
temp_sentences.append(paragraph[i]) # 1st sentence
temp_sent_seq_length += len(paragraph[i])

print('current seq length:', temp_sent_seq_length)
print('# of sentences in the paragraph: {}'.format(len(paragraph)))
print('temp_sentences len: {}'.format(len(temp_sentences)), '\n', temp_sentences)

if i == len(paragraph) - 1 or temp_sent_seq_length >= target_seq_len:
    print('Is it the last sentence of the paragraph \nor current chunk longer than target_seq_len?', True)

current seq length: 311
# of sentences in the paragraph: 6
temp_sentences len: 4 
 [['▁그러나', '▁이것은', '▁공', '화', '당', '과', '▁미국의', '▁유대', '인', '▁단', '체의', '▁반', '발', '을', '▁일으', '켰', '다', '.', '▁1979', '년', '▁백', '악', '관', '에서', '▁양', '국', '▁간의', '▁평화', '조', '약', '으로', '▁이끌', '어졌다', '.', '▁또한', '▁소련', '과', '▁제', '2', '차', '▁전략', '▁무', '기', '▁제한', '▁협', '상에', '▁조', '인', '했다', '.'], ['▁카', '터', '는', '▁1970', '년대', '▁후반', '▁당시', '▁대한민국', '▁등', '▁인', '권', '▁후', '진', '국의', '▁국민', '들의', '▁인', '권을', '▁지', '키', '기', '▁위해', '▁노력', '했으며', ',', '▁취임', '▁이후', '▁계속', '해서', '▁도', '덕', '정', '치를', '▁내', '세', '웠다', '.'], ['▁그러나', '▁주', '▁이란', '▁미국', '▁대사', '관', '▁인', '질', '▁사건', '에서', '▁인', '질', '▁구', '출', '▁실패', '를', '▁이유로', '▁1980', '년', '▁대통령', '▁선거', '에서', '▁공', '화', '당의', '▁로', '널', '드', '▁레이', '건', '▁후보', '에게', '▁', '져', '▁결국', '▁재', '선에', '▁실패', '했다', '.', '▁또한', '▁임', '기', '▁말', '기에', '▁터', '진', '▁소련', '의', '▁아', '프가', '니', '스탄', '▁침공', '▁사건', '으로', '▁인해', '▁1980', '년', '▁하계', '▁올림픽', '에', '▁반',

- tokenA

In [40]:
if 0 < len(temp_sentences):
    a_end = 1
    if 1 < len(temp_sentences):
        a_end = np.random.randint(1, len(temp_sentences))
        
    tokenA = []
    for j in range(a_end):
        tokenA.extend(temp_sentences[j]) # convert sentence to segment
        
print('tokenA consists of {} sentences'.format(a_end), '\n', tokenA)

tokenA consists of 3 sentences 
 ['▁그러나', '▁이것은', '▁공', '화', '당', '과', '▁미국의', '▁유대', '인', '▁단', '체의', '▁반', '발', '을', '▁일으', '켰', '다', '.', '▁1979', '년', '▁백', '악', '관', '에서', '▁양', '국', '▁간의', '▁평화', '조', '약', '으로', '▁이끌', '어졌다', '.', '▁또한', '▁소련', '과', '▁제', '2', '차', '▁전략', '▁무', '기', '▁제한', '▁협', '상에', '▁조', '인', '했다', '.', '▁카', '터', '는', '▁1970', '년대', '▁후반', '▁당시', '▁대한민국', '▁등', '▁인', '권', '▁후', '진', '국의', '▁국민', '들의', '▁인', '권을', '▁지', '키', '기', '▁위해', '▁노력', '했으며', ',', '▁취임', '▁이후', '▁계속', '해서', '▁도', '덕', '정', '치를', '▁내', '세', '웠다', '.', '▁그러나', '▁주', '▁이란', '▁미국', '▁대사', '관', '▁인', '질', '▁사건', '에서', '▁인', '질', '▁구', '출', '▁실패', '를', '▁이유로', '▁1980', '년', '▁대통령', '▁선거', '에서', '▁공', '화', '당의', '▁로', '널', '드', '▁레이', '건', '▁후보', '에게', '▁', '져', '▁결국', '▁재', '선에', '▁실패', '했다', '.', '▁또한', '▁임', '기', '▁말', '기에', '▁터', '진', '▁소련', '의', '▁아', '프가', '니', '스탄', '▁침공', '▁사건', '으로', '▁인해', '▁1980', '년', '▁하계', '▁올림픽', '에', '▁반', '공', '국', '가', '들의', '▁보이', '콧', '을', '▁내', '세', '웠다',

- tokenB random sentence with 1/2 probability

In [49]:
tokenB = []
if len(temp_sentences) == 1 or np.random.uniform() < 0.5:
    is_next = False
    tokenB_len = target_seq_len - len(tokenA) # minumum len of tokenB
    print('tokenB avaliable length: {}'.format(tokenB_len))
    
    # choose sentence from other paragraph
    random_para_idx = para_idx
    while para_idx == random_para_idx:
        random_para_idx = np.random.randint(0, len(paragraph_ls))
    # this is random paragraph    
    random_paragraph = paragraph_ls[random_para_idx]
    
    # add a series of sentences from random paragragraph
    random_start = np.random.randint(0, len(random_paragraph))
    for j in range(random_start, len(random_paragraph)):
        tokenB.extend(random_paragraph[j])
        if len(tokenB) > tokenB_len:
            break
print('tokenB len {} should be longer than {}, \nor truncate segments'.format(len(tokenB), tokenB_len))
print(tokenB[:10])

tokenB avaliable length: 92
tokenB len 97 should be longer than 92, 
or truncate segments
['▁1976', '년', '▁대통령', '▁선거', '에', '▁민주', '당', '▁후보', '로', '▁출']


- tokenB the sentences right after tokenA

In [50]:
is_next = 1
for j in range(a_end, len(temp_sentences)):
    tokenB.extend(temp_sentences[j])
print('the end of the tokenA: \n{}'.format(tokenA[-10:]))
print('the beggining of the tokenB: \n{}'.format(tokenB[:10]))

the end of the tokenA: 
['국', '가', '들의', '▁보이', '콧', '을', '▁내', '세', '웠다', '.']
the beggining of the tokenB: 
['▁1976', '년', '▁대통령', '▁선거', '에', '▁민주', '당', '▁후보', '로', '▁출']


- truncate long segmentsA, B

In [84]:
truncate_token(tokenA, tokenB, max_seq)

max token 253
total_len 253 = 127 + 126


In [104]:
assert 0 < len(tokenA)
assert 0 < len(tokenB)
tokens = ['[CLS]'] + tokenA + ['[SEP]'] + tokenB + ['[SEP]']
segment = [0]*(len(tokenA)  + 2) + [1]*(len(tokenB) + 1)

In [111]:
print(segment)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


---
# make pre train data

In [180]:
import os

def make_pretrain_data(vocab, in_file, out_file, count, n_seq, mask_prob):
    """
    read text and return train data set format
    """
    vocab_list = []
    for id_ in range(vocab.get_piece_size()):
        if not vocab.is_unknown(id_):
            vocab_list.append(vocab.id_to_piece(id_))
    
    paragraph_ls = []
    with open(in_file, 'r') as in_f:
        paragraph = []
        for i, sent in enumerate(in_f):
            sent = sent.strip()
            
            ## blank means end of the paragraph
            if sent == '':
                # if not the beggining of the paragraph
                # it is the end of the paragraph
                if 0 < len(paragraph):
                    paragraph_ls.append(paragraph)
                    paragraph = [] # generate new paragraph list
                    # check if exceeding 100 thaousand paragraphs
                    if 1e+5 < len(paragraph_ls): 
                        break 
                        
            ## subwords in list is part of semantic token
            # eg. ['▁지','미','▁카','터']
            else:
                pieces = vocab.encode_as_pieces(sent)
                if 0 < len(pieces):
                    paragraph.append(pieces)
        if paragraph:
            paragraph_ls.append(paragraph)
    # masking def: create_pretrain_mask
    for index in range(count):
        output = out_file.format(index)
        if os.path.isfile(output):
            continue
        with open(output, 'w') as out_f:
            for i, paragraph in enumerate(paragraph_ls):
                print(paragraph)
                masking_info = create_pretrain_instances(paragraph_ls, i, paragraph, n_seq, mask_prob, vocab_list)
                for elem in masking_info:
                    out_f.write(json.dumps(elem))
                    out_f.write('\n')    

- example code

In [132]:
# test data
#in_PATH = '/home/henry/Documents/wrapper/source/kowiki.txt'
in_PATH = '/home/henry/Documents/wrapper/source/kowiki_sample.txt' # sample text
sentences = []
with open(in_PATH, 'r') as in_f:
    for i, sent in enumerate(in_f):
        # '': paragraph delimiter
        if i == 7:
            sentences.append('')
        if i == 13:
            break
        sentences.append(sent)

In [133]:
len(sentences), sentences[:4]

(10,
 ['지미 카터\n',
  '제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.\n',
  '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n',
  '1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n'])

In [134]:
paragraph_ls = []
paragraph = []
for i, sent in enumerate(sentences):
    sent = sent.strip()
    ## blank means end of paragraph
    if sent == '':
        # if not the beggining of sentence
        # it is the end of the paragraph
        # generate new doc list
        if 0 < len(paragraph):
            paragraph_ls.append(paragraph)
            paragraph = []
            # check if 100thaousand paragraph
            if 1e+5 < len(paragraph_ls):
                break
                
    ## subwords in list is part of semantic token
    # eg. ['▁지','미','▁카','터']
    else:
        pieces = vocab.encode_as_pieces(sent)
        if 0 < len(pieces):
            paragraph.append(pieces)
if paragraph:
    paragraph_ls.append(paragraph)

In [135]:
print('num of paragraph:', len(paragraph_ls))

num of paragraph: 2


In [136]:
print('1st paragraph - 1st sentence')
print(paragraph_ls[0][0])

1st paragraph - 1st sentence
['▁지', '미', '▁카', '터']


In [137]:
print('1st paragraph - 2nd sentence')
print(paragraph_ls[1][0])

1st paragraph - 2nd sentence
['▁그러나', '▁이것은', '▁공', '화', '당', '과', '▁미국의', '▁유대', '인', '▁단', '체의', '▁반', '발', '을', '▁일으', '켰', '다', '.', '▁1979', '년', '▁백', '악', '관', '에서', '▁양', '국', '▁간의', '▁평화', '조', '약', '으로', '▁이끌', '어졌다', '.', '▁또한', '▁소련', '과', '▁제', '2', '차', '▁전략', '▁무', '기', '▁제한', '▁협', '상에', '▁조', '인', '했다', '.']


In [138]:
print('2nd paragraph - 1st sentence')
print(paragraph_ls[1][1])

2nd paragraph - 1st sentence
['▁카', '터', '는', '▁1970', '년대', '▁후반', '▁당시', '▁대한민국', '▁등', '▁인', '권', '▁후', '진', '국의', '▁국민', '들의', '▁인', '권을', '▁지', '키', '기', '▁위해', '▁노력', '했으며', ',', '▁취임', '▁이후', '▁계속', '해서', '▁도', '덕', '정', '치를', '▁내', '세', '웠다', '.']


In [182]:
in_PATH = '/home/henry/Documents/wrapper/source/kowiki_sample.txt' 
out_PATH = '/home/henry/Documents/wrapper/source/out_kowiki_sample' + '_{}.json'

count = 1
n_seq = 10
mask_prob = 0.15

make_pretrain_data(vocab, in_PATH, out_PATH, count, n_seq, mask_prob)

In [188]:
# result
pretrain_output = []
with open('/home/henry/Documents/wrapper/source/out_kowiki_sample_0.json', 'r') as f:
    for line in f:
        temp = json.loads(line)
        pretrain_output.append(temp)

In [193]:
len(pretrain_output), pretrain_output[0].keys()

(2, dict_keys(['tokens', 'segment', 'is_next', 'mask_idx', 'mask_label']))

In [197]:
for k, v in pretrain_output[0].items():
    print(k)
    print(v)
    print()

tokens
['[CLS]', '▁지', '미', '▁카', '터', '▁제임스', '▁얼', '▁"', '지', '미', '"', '▁카', '터', '▁주', '니어', '(,', '▁192', '4', '년', '▁10', '월', 'Θ', '▁M', '▁2007', '▁)', 'び', '▁민주', '당', '▁출신', '▁미국', 'г', '9', '번째', '▁대통령', '▁(19', '7', '7', '년', '▁~', '▁1981', '년', ')', '이다', '.', '[SEP]', '▁지', '▁초등학교이다', '▁카', '터', '는', '▁조지', '아', '주', '▁섬', '터', '▁카', '운', '티', '▁플', '레', '인', '스', '▁마을', '에서', '▁태어났다', '.', '▁조지', '아', '▁공', '과', '대학교', '를', '▁졸업', '하였다', '.', '▁그', '▁후', '▁해', '군에', '▁들어가', '▁전', '함', '·', '慚', '자', '력', '眠', '잠', '수', '함', '의', '▁승', '무', '원으로', '녘', '하였다', '.', '▁195', '3', '년', '懸', '▁해군', '▁대', '위로', '▁예', '편', '하였고', '▁이후', '▁땅', '콩', '·', '면', '화', '▁등을', '▁가', '꿔', '▁많은', '▁돈', '을', '▁벌', '었다', '.', '▁그의', '▁별', '명이', '▁"', '땅', '콩', '▁농', '부', '"', '▁(', 'P', 'e', '岳', '따', '▁F', 'ar', 'm', 'er', ')', '로', '▁알려', '졌다', '.', '▁196', '2', '년', '▁조지', '아', '▁주', '▁상', '원', '環', '▁선거', '에서', '▁낙', '선', '하나', '▁그', '▁선거', '가', '▁부정', '선거', '▁', '였', '음을', '▁입', '증', '하

---
# Pretrain Data

In [198]:
class PretrainDataset(Dataset):
    def __init__(self, vocab, infile):
        self.vocab = vocab
        self.labels_cls = []
        self.labels_lm = []
        self.sentences = []
        self.segments = []
        
        with open(infile, 'r') as f:
            for i, line in enumerate(f):
                instance = json.loads(line)
                self.labels_cls.append(instance['is_next'])
                